## Using OpenAI Tool Calling to create this agent. 

Initially will create without memory, but eventually we will add memory in. Memory modules play a critical role in AI agents. A memory module can essentially be thought of as a store of the agent’s internal logs as well as interactions with a user. 

In [1]:
# install all the relevant packages
# pip install -U langchain-openai
# pip install langchain-agents
# pip install langchain-community
# pip install langchain-experimental
# pip show langchain

In [99]:
## dont forget to remove you API Key 
import os
os.environ['OPENAI_API_KEY']="YOUR_API_KEY"

### Load the LLM

In [100]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [101]:
import sys
sys.path.insert(0, '/path/to/module')

### Define Tools

In [102]:
from langchain.agents import tool

In [103]:
@tool
def get_word_length(word: str) ->int:
    """Returns the length of the word."""
    return len(word)

get_word_length("abc")

3

In [104]:
tools = [get_word_length]

### Create Prompt

As OpenAI Function Calling is finetuned for tool usage, we don't have to provide instructions on how to reason, or how to output format. We will have two input variables: input and agent_scratchpad. 
"input" is a string containing the user objective. "agent_scratchpad" is a sequence of messages that contains the previous agent tool invocations and the corresponding tool outputs.

In [105]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [106]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

### Binding tools to LLM

OpenAI tool calling LLMs which take tools as a separate argument and have been specifically trained to know when to invoke those tools. To pass in our tools to the agent, we need to format them to OpenAI tool format and pass them to our model. (By binding the functions, we're making sure that they are passed in each time the model in invoked.)

In [107]:
llm_with_tools = llm.bind_tools(tools)

### Creating the Agent

Import the last two utility functions: a component for formatting intermediate steps(agent action, tool, output pairs) to input messages that can be sent to the model, and a component for converting the output message into an agent action/agent finish

In [108]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [109]:
from langchain.agents import AgentExecutor

agent_executer = AgentExecutor(agent = agent, tools = tools, verbose = True)

In [111]:
list(agent_executer.stream({"input": "How many letters in the word eudca"}))



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'eudca'}`


5The word "eudca" has 5 letters.

> Finished chain.


[{'actions': [ToolAgentAction(tool='get_word_length', tool_input={'word': 'eudca'}, log="\nInvoking: `get_word_length` with `{'word': 'eudca'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_3eREewInqqj3ySQZcfyGLY3W', 'function': {'arguments': '{"word":"eudca"}', 'name': 'get_word_length'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'}, id='run-0498c24d-acfa-491a-b9f3-1d16eb6b3c76', tool_calls=[{'name': 'get_word_length', 'args': {'word': 'eudca'}, 'id': 'call_3eREewInqqj3ySQZcfyGLY3W'}], tool_call_chunks=[{'name': 'get_word_length', 'args': '{"word":"eudca"}', 'id': 'call_3eREewInqqj3ySQZcfyGLY3W', 'index': 0}])], tool_call_id='call_3eREewInqqj3ySQZcfyGLY3W')],
  'messages': [AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_3eREewInqqj3ySQZcfyGLY3W', 'function': {'arguments': '{"word":"eudca"}', 'name': 'get_word_length'}, 'type': 'function'}]}, response_metadata

In [112]:
llm.invoke("How many letters in the word educa")

AIMessage(content='5', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 15, 'total_tokens': 16}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a0f98306-4588-4a0f-ab07-8c416a4e6916-0', usage_metadata={'input_tokens': 15, 'output_tokens': 1, 'total_tokens': 16})

### Adding Memory 

This agent is stateless - it doesn't remember anything about previous interactions. In order to do this, we need to do two things:

Add a place for memory variables to go in the prompt and keep track of the chat history

In [114]:
#### adding place for memory in the prompt. By adding a placeholder for messages with the key "chat_history"

from langchain_core.prompts import MessagesPlaceholder

MEMORY_KEY ="chat_history"
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but bad at calculating lengths of words.",
        ),
        MessagesPlaceholder(variable_name = MEMORY_KEY),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name = "agent_scratchpad"),
    ]
)

In [115]:
#Setting up a list to track the chat history

from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

In [118]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x:x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

agent_executer = AgentExecutor( agent=agent, tools=tools, verbose =True)

In [119]:
input1 = "How many letters in the word educa?"
result = agent_executer.invoke({"input": input1, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)
agent_executer.invoke({"input": "is that a real word?", "chat_history":chat_history})



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educa'}`


5There are 5 letters in the word "educa".

> Finished chain.


> Entering new AgentExecutor chain...
Yes, "educa" is not a real word in English. It seems like a shortened or incomplete version of the word "education".

> Finished chain.


{'input': 'is that a real word?',
 'chat_history': [HumanMessage(content='How many letters in the word educa?'),
  AIMessage(content='There are 5 letters in the word "educa".')],
 'output': 'Yes, "educa" is not a real word in English. It seems like a shortened or incomplete version of the word "education".'}